<a href="https://colab.research.google.com/github/CharuRathee/Aircraft_detection/blob/main/Copy_of_Military_Aircraft_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environment and Library Checks

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import shutil
import cv2
import os
from tqdm import tqdm
MODE = 'valid'# train, valid

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.7/778.7 kB 3.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.23 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 29.9/78.2 GB disk)


In [ ]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    print('CUDA is available. GPU will be used.')
else:
    print('CUDA is not available. CPU will be used.')

CUDA is available. GPU will be used.


# Defining Classes for Military Aircraft Detection

In [ ]:
classes = np.array(['A10','A400M','AG600','AV8B',
        'B1','B2','B52','Be200',
        'C130','C2','C17','C5','E2','E7','EF2000',
        'F117','F14','F15','F16','F18','F22','F35','F4',
        'JAS39','MQ9','Mig31','Mirage2000','P3','RQ4','Rafale',
        'SR71','Su34','Su57',
        'Tu160','Tu95','Tornado','Tu22M',
        'U2','US2', 'V22','XB70','YF23','Vulcan','J20',
        'KC135', 'J10', 'Su25', 'Su24','H6', 'JF17']
)
print(len(classes))

50


# Importing Kaggle Files and Preparing Dataset Structure

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"charurathee","key":"04c3cf646f58ad25a997a23ee66c3da0"}'}

In [ ]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 May 28 12:18 kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets list

ref                                                    title                                         size  lastUpdated          downloadCount  voteCount  usabilityRating  
-----------------------------------------------------  -------------------------------------------  -----  -------------------  -------------  ---------  ---------------  
rahulvyasm/netflix-movies-and-tv-shows                 Netflix Movies and TV Shows                    1MB  2024-04-10 09:48:38          25166        537  1.0              
mayankanand2701/zomato-stock-price-dataset             Zomato Stock Price Dataset 📊🍗🏨🍴               13KB  2024-05-23 06:42:34            986         25  1.0              
zeesolver/spotfy                                       Spotify Songs Album                           47KB  2024-05-10 09:21:57           1757         26  1.0              
prasad22/weather-data                                  Weather Data                                  43MB  2024-05-18 14:27:35           165

In [ ]:
!kaggle datasets download -d a2015003713/militaryaircraftdetectiondataset

Dataset URL: https://www.kaggle.com/datasets/a2015003713/militaryaircraftdetectiondataset
License(s): unknown
100% 12.8G/12.8G [02:27<00:00, 63.6MB/s]
100% 12.8G/12.8G [02:27<00:00, 93.4MB/s]


In [ ]:
# Unzip the dataset
!unzip militaryaircraftdetectiondataset.zip -d /content/militaryaircraftdetectiondataset

Streaming output truncated to the last 5000 lines.
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e17de50506df034cd8aaab6738df4e.csv  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e17de50506df034cd8aaab6738df4e.jpg  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e46b56fa6f12cbd98079d5ab1dea0d.csv  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e46b56fa6f12cbd98079d5ab1dea0d.jpg  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e754f7e2f6acb9eeff66239f0e2e4e.csv  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e754f7e2f6acb9eeff66239f0e2e4e.jpg  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e820543904aecbe01e2bd4fea8b9c7.csv  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4e820543904aecbe01e2bd4fea8b9c7.jpg  
  inflating: /content/militaryaircraftdetectiondataset/dataset/d4ecb26d0ad0d110a1de6303d67e657a.csv  
  inflating: /content/militarya

In [ ]:
!kaggle datasets download -d a2015003713/yolov7-military-plane

# Unzip the Downloaded Dataset
!unzip yolov7-military-plane.zip

# Verify the Contents
import os
print(os.listdir('/content'))

Dataset URL: https://www.kaggle.com/datasets/a2015003713/yolov7-military-plane
License(s): unknown
 97% 266M/274M [00:03<00:00, 96.4MB/s]
100% 274M/274M [00:03<00:00, 90.4MB/s]
Archive:  yolov7-military-plane.zip
  inflating: rtdetr-l-best.pt        
  inflating: rtdetr-l-best_46cls.pt  
  inflating: rtdetr-l-best_49cls.pt  
  inflating: yolov7-w6-best.pt       
  inflating: yolov8-m-best.pt        
['.config', 'kaggle.json', 'rtdetr-l-best_46cls.pt', 'rtdetr-l-best_49cls.pt', 'rtdetr-l-best.pt', 'yolov8-m-best.pt', 'yolov7-w6-best.pt', 'militaryaircraftdetectiondataset.zip', 'militaryaircraftdetectiondataset', 'yolov7-military-plane.zip']


# Setting up Directory Structure

In [ ]:
# Path to the unzipped dataset
dataset_path = '/content/militaryaircraftdetectiondataset/dataset'

# Collect CSV and JPG paths
csv_paths = glob.glob(os.path.join(dataset_path, '*.csv'))
jpg_paths = glob.glob(os.path.join(dataset_path, '*.jpg'))
csv_paths.sort()
jpg_paths.sort()
print('number of images:', len(csv_paths))

# Create necessary directories in the Colab environment
os.makedirs('/content/ultralytics/data/train/images', exist_ok=True)
os.makedirs('/content/ultralytics/data/train/labels', exist_ok=True)

os.makedirs('/content/ultralytics/data/valid/images', exist_ok=True)
os.makedirs('/content/ultralytics/data/valid/labels', exist_ok=True)

os.makedirs('/content/ultralytics/data/test/images', exist_ok=True)
os.makedirs('/content/ultralytics/data/test/labels', exist_ok=True)

number of images: 14462


# Preparing Dataset Annotations and File Structure

In [ ]:
MODE = 'train'
for i, (csv_path, jpg_path) in enumerate(tqdm(zip(csv_paths, jpg_paths))):
    annotations = np.array(pd.read_csv(csv_path))

    if os.path.basename(csv_path)[0] in ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'f']:
        if MODE == 'train':
            jpg_file_path = '/content/ultralytics/data/train/images/' + os.path.basename(jpg_path)
            txt_file_path = '/content/ultralytics/data/train/labels/' + os.path.basename(csv_path)[:-4] + '.txt'
            shutil.copy(jpg_path, jpg_file_path)
        else:
            continue
    elif os.path.basename(csv_path)[0] in ['c', 'd', 'e']:
        name = os.path.splitext(os.path.basename(jpg_path))[0]
        new_name = ''.join(np.random.permutation(list(name)))
        jpg_file_path = '/content/ultralytics/data/valid/images/' + new_name + '.jpg'
        txt_file_path = '/content/ultralytics/data/valid/labels/' + new_name + '.txt'
        shutil.copy(jpg_path, jpg_file_path)
    else:
        continue

    with open(txt_file_path, mode='w') as f:
        try:
            for annotation in annotations:
              width = annotation[1]
              height = annotation[2]
              class_name = annotation[3]
              xmin = annotation[4]
              ymin = annotation[5]
              xmax = annotation[6]
              ymax = annotation[7]
              x_center = 0.5 * (xmin + xmax)
              y_center = 0.5 * (ymin + ymax)
              b_width = xmax - xmin
              b_height = ymax - ymin

              class_indices = np.where(classes == class_name)
              if len(class_indices[0]) == 0:
                print(f"Error: Class name '{class_name}' not found in classes array.")
                continue  # Skip this annotation or handle the error appropriately

              class_num = class_indices[0][0]
              output_string = '{} {} {} {} {}\n'.format(class_num, xmin, ymin, b_width, b_height)

              output_string = '{} {} {} {} {}\n'.format(
                    class_num,
                    x_center / width,
                    y_center / height,
                    b_width / width,
                    b_height / height
              )
              f.write(output_string)
        except:
          print(txt_file_path)
          raise

print('test:', len(glob.glob('/content/ultralytics/data/test/labels/*.txt')))
print('valid:', len(glob.glob('/content/ultralytics/data/valid/labels/*.txt')))
print('train:', len(glob.glob('/content/ultralytics/data/train/labels/*.txt')))

14462it [02:00, 119.91it/s]

test: 0
valid: 2703
train: 11759


# Generating YAML Configuration File

In [ ]:
# Generate the classes string for the YAML file
classes_string = ', '.join([f'"{c}"' for c in classes])


yaml_content = f"""
train: /content/ultralytics/data/train
val: /content/ultralytics/data/valid
test: /content/ultralytics/data/test
nc: {len(classes)}
names: [{classes_string}]
"""

yaml_file_path = '/content/ultralytics/data/mad.yaml'
os.makedirs(os.path.dirname(yaml_file_path), exist_ok=True)
with open(yaml_file_path, mode='w') as f:
    f.write(yaml_content.strip())

print(yaml_content)


train: /content/ultralytics/data/train
val: /content/ultralytics/data/valid
test: /content/ultralytics/data/test
nc: 50
names: ["A10", "A400M", "AG600", "AV8B", "B1", "B2", "B52", "Be200", "C130", "C2", "C17", "C5", "E2", "E7", "EF2000", "F117", "F14", "F15", "F16", "F18", "F22", "F35", "F4", "JAS39", "MQ9", "Mig31", "Mirage2000", "P3", "RQ4", "Rafale", "SR71", "Su34", "Su57", "Tu160", "Tu95", "Tornado", "Tu22M", "U2", "US2", "V22", "XB70", "YF23", "Vulcan", "J20", "KC135", "J10", "Su25", "Su24", "H6", "JF17"]



# Training RT-DETR Model

In [ ]:
MODE = 'train'
if MODE == 'train':
    from ultralytics import RTDETR
    import os

    # Set the path to the pretrained model
    pretrained_model_path = '/content/rtdetr-l.pt'
    # Set the path to the data YAML file
    data_yaml_path = '/content/ultralytics/data/mad.yaml'

    # Initialize the RTDETR model
    model = RTDETR(model='rtdetr-l.pt')

    # Train the model
    result = model.train(
        name='RT-DETR_001',
        epochs=5,
        imgsz=640,
        batch=4,
        # optimizer='auto',

        # augmentation
        close_mosaic=0,
        hsv_h=0.02,
        hsv_s=0.75,
        hsv_v=0.45,
        degrees=0.1,
        translate=0.25,
        scale=0.55,
        shear=0.1,
        perspective=0.0,
        flipud=0.1,
        fliplr=0.5,
        mosaic=1.0,
        mixup=0.5,

        pretrained=pretrained_model_path,
        data=data_yaml_path,
    )

100%|██████████| 63.4M/63.4M [00:00<00:00, 189MB/s]


Ultralytics YOLOv8.2.23 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=rtdetr-l.pt, data=/content/ultralytics/data/mad.yaml, epochs=5, time=None, patience=100, batch=4, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=RT-DETR_001, exist_ok=False, pretrained=/content/rtdetr-l.pt, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels

100%|██████████| 755k/755k [00:00<00:00, 14.8MB/s]


Overriding model.yaml nc=80 with nc=50
WARNING ⚠️ no model scale passed. Assuming scale='l'.

                   from  n    params  module                                       arguments                     
  0                  -1  1     25248  ultralytics.nn.modules.block.HGStem          [3, 32, 48]                   
  1                  -1  6    155072  ultralytics.nn.modules.block.HGBlock         [48, 48, 128, 3, 6]           
  2                  -1  1      1408  ultralytics.nn.modules.conv.DWConv           [128, 128, 3, 2, 1, False]    
  3                  -1  6    839296  ultralytics.nn.modules.block.HGBlock         [128, 96, 512, 3, 6]          
  4                  -1  1      5632  ultralytics.nn.modules.conv.DWConv           [512, 512, 3, 2, 1, False]    
  5                  -1  6   1695360  ultralytics.nn.modules.block.HGBlock         [512, 192, 1024, 5, 6, True, False]
  6                  -1  6   2055808  ultralytics.nn.modules.block.HGBlock         [1024, 192, 1024, 5,

100%|██████████| 6.23M/6.23M [00:00<00:00, 72.4MB/s]
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


...

val: WARNING ⚠️ /content/ultralytics/data/valid/images/2b0c547808b4c6d4661702c9b6db2419.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/ultralytics/data/valid/images/32810c8a3cbced9653e31e128adee7a5.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/ultralytics/data/valid/images/47d815ecf0a47ad7f84b97c30d123b59.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/ultralytics/data/valid/images/47feedf6811db21bd0951916bcb7c59b.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/ultralytics/data/valid/images/5a704a22ab0c7909368d935d6f55f148.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/ultralytics/data/valid/images/788fff17f862dd08686910e23fc3ef17.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/ultralytics/data/valid/images/7d1798430ae56bd524c8c8d6090628e6.jpg: corrupt JPEG restored and saved
val: WARNING ⚠️ /content/ultralytics/data/valid/images/7e2eaec1260a3db7f3a0a2ff3fad5132.jpg: corrupt JPEG restored and saved


Plotting labels to runs/detect/RT-DETR_001/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000185, momentum=0.9) with parameter groups 143 weight(decay=0.0), 206 weight(decay=0.0005), 226 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/RT-DETR_001
Starting training for 5 epochs...

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/2939 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
        1/5      4.29G     0.5014      1.865     0.3755          3        640: 100%|██████████| 2939/2939 [37:32<00:00,  1.30it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 338/338 [03:36<00:00,  1.56it/s]


                   all       2702       4582       0.55      0.047     0.0145     0.0125

      Epoch    GPU_mem  giou_loss   cls_loss    l1_loss  Instances       Size


  0%|          | 0/2939 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: grid_sampler_2d_backward_cuda does not have a deterministic implementation, but you set 'torch.use_deterministic_algorithms(True, warn_only=True)'. You can file an issue at https://github.com/pytorch/pytorch/issues to help us prioritize adding deterministic support for this operation. (Triggered internally at ../aten/src/ATen/Context.cpp:83.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
        2/5      4.34G     0.3332      1.699     0.2304          8        640: 100%|██████████| 2939/2939 [37:15<00:00,  1.31it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  12%|█▏        | 39/338 [00:21<01:58,  2.52it/s]

# Validating Trained Weights with YOLO

In [ ]:
# validate trained weights
!yolo val \
model='/content/rtdetr-l.pt' \
data='/content/ultralytics/data/mad.yaml' \
augment \
batch=16 \
imgsz=840

# Listing PNG Files

In [ ]:
import os
from glob import glob

# List all PNG files in the directory
png_files = glob('/content/runs/detect/val/*.png')
print(png_files)

# Visualizing Detected Objects in Images with OpenCV and Matplotlib

In [ ]:
import cv2
import glob  # Import the glob module

import matplotlib.pyplot as plt

# Loop through images in the specified directory
for path in sorted(glob.glob('/content/runs/detect/val/*.png')):
    # Read the image
    image = cv2.imread(path)

    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.figure(figsize=(20,20))
    plt.imshow(image_rgb)
    plt.axis('off')  # Turn off axis
    plt.show()

# Image Visualization from Detection Results

In [ ]:
# Loop through images in the specified directory
for path in sorted(glob.glob('/content/runs/detect/val/*.jpg')):
    # Read the image
    image = cv2.imread(path)

    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image
    plt.figure(figsize=(20, 20))
    plt.imshow(image_rgb)
    plt.axis('off')  # Turn off axis
    plt.show()